# List Intermediaries by Country and Number of Transfers

In [1]:
import json, pandas as pd, requests, random

In [2]:
raw_transfers = requests.get("https://tsosi.org/api/transfers/all?format=json").json()

In [3]:
## transfrom to df
df_transfers = pd.json_normalize(raw_transfers)

In [4]:
df_transfers.columns

Index(['id', 'emitter_id', 'recipient_id', 'agent_id', 'amount', 'currency',
       'description', 'date_clc.value', 'date_clc.precision',
       'amounts_clc.CAD', 'amounts_clc.DKK', 'amounts_clc.EUR',
       'amounts_clc.GBP', 'amounts_clc.USD', 'amounts_clc'],
      dtype='object')

In [5]:
# create a df for only intermediaries with the columns we need
df_intermed = df_transfers[ df_transfers["agent_id"].notna()][["emitter_id", "recipient_id", "agent_id", "date_clc.value", "amounts_clc.EUR"]]

In [6]:
df_intermed.sample(2)

,emitter_id,recipient_id,agent_id,date_clc.value,amounts_clc.EUR
2511,1aaa5b12-01fc-4986-96d5-10f98f427ab2,c584df0e-75ea-44ee-a0b9-3a4b5c0706be,ae82f521-d130-4d64-94f5-ac534b8ad569,2024-05-14,NaN
1984,aad571a4-4bf0-4c9a-a8b0-73f052db3afb,814124ee-2515-4a27-aed9-3175eeddf6cc,30469b19-40ab-403b-a7c9-59d1301ca154,2024-01-01,NaN


In [7]:
# add a column with the sum of transfers made by one intermediary (when we have the amount)
df_intermed['total_sum'] = df_intermed.groupby('agent_id')['amounts_clc.EUR'].transform('sum')

In [8]:
df_intermed.sample(1)

,emitter_id,recipient_id,agent_id,date_clc.value,amounts_clc.EUR,total_sum
2599,43bce5cd-f5c2-4adf-8a2b-30fe18f64c36,814124ee-2515-4a27-aed9-3175eeddf6cc,63455bff-a695-4d4e-b9fd-ac4f100d4de8,2024-07-01,NaN,0.0


In [9]:
# group by intermediary and add columns for number of emitters involved, number of transfers, and the last column of the sum of the transfers
df_intermed_meta = df_intermed.groupby('agent_id').agg(
        nb_emitters=('emitter_id', 'nunique'),
        nb_transfers=('date_clc.value', 'count'),
        transfer_sum=('total_sum', 'last')
).reset_index()

In [10]:
df_intermed_meta.columns

Index(['agent_id', 'nb_emitters', 'nb_transfers', 'transfer_sum'], dtype='object')

In [11]:
## get a table with all entities

In [12]:
raw_entities = requests.get("https://tsosi.org/api/entities/all?format=json").json()
df_entities = pd.json_normalize(raw_entities)
df_entities.columns

Index(['id', 'name', 'short_name', 'country', 'identifiers', 'coordinates',
       'logo', 'icon', 'is_recipient', 'is_partner'],
      dtype='object')

In [13]:
df_results = df_intermed_meta.merge(
    df_entities[['id', 'name', 'country', 'identifiers']], left_on='agent_id', right_on='id', how = 'left')

In [14]:
df_results = df_results[['name', 'country', 'nb_emitters', 'nb_transfers', 'transfer_sum', 'identifiers']].sort_values(by=["country"])

In [15]:
df_results.head()

,name,country,nb_emitters,nb_transfers,transfer_sum,identifiers
38,Council of Australasian University Librarians,AU,16,23,0.0,"[{'registry': 'ror', 'value': '00aa7w613'}, {'..."
34,Bibliothèque Interuniversitaire de la Communau...,BE,4,8,0.0,"[{'registry': 'wikidata', 'value': 'Q76612327'}]"
17,Elektron VZW,BE,9,25,0.0,"[{'registry': 'wikidata', 'value': 'Q27991319'}]"
0,Ontario Council of University Libraries,CA,13,22,0.0,"[{'registry': 'wikidata', 'value': 'Q30289390'..."
30,Council of Atlantic University Libraries,CA,7,11,0.0,"[{'registry': 'wikidata', 'value': 'Q76612380'}]"


In [16]:
df_results.to_csv("2025-05-25--intermediaries-metadata.csv", index=False)